<a href="https://colab.research.google.com/github/angelraphael123/book_reccomentation_knn/blob/main/Copy_of_fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'


--2024-08-07 06:00:29--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.6’

book-crossings.zip. 100%[===================>]  24.88M   103MB/s    in 0.2s    

2024-08-07 06:00:29 (103 MB/s) - ‘book-crossings.zip.6’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace BX-Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace BX-Users.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import normalize

# Normalize the data (normalize items)
user_item_matrix_normalized = normalize(user_item_matrix.T)  # Transpose to normalize items (books)

print(user_item_matrix.shape)
print(user_item_matrix_normalized.shape)

# Build the KNN model with cosine metric
knn = NearestNeighbors(n_neighbors=6, metric='cosine', algorithm='brute')
knn.fit(user_item_matrix_normalized)  # Fit the KNN model on normalized item vectors


(888, 731)
(731, 888)


NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

In [ ]:
def get_recommends(book_title):
    if not book_title:
        return [book_title, []]

    # Find the ISBN for the given book title
    book_isbn = df_books[df_books['title'] == book_title]['isbn'].values
    if len(book_isbn) == 0:
        return [book_title, []]

    book_isbn = book_isbn[0]

    # Check if the book_isbn is in the user-item matrix
    if book_isbn not in user_item_matrix.columns:
        return [book_title, []]

    # Get the index of the book_isbn in the user-item matrix
    book_index = user_item_matrix.columns.get_loc(book_isbn)

    # Get the book's vector
    book_vector = user_item_matrix_normalized[book_index].reshape(1, -1)  # Reshape to 2D

    print(f"Book Index: {book_index}")
    print(f"Book Vector Shape: {book_vector.shape}")

    # Predict distances to all other books
    distances, indices = knn.kneighbors(book_vector)

    print(f"Distances: {distances}")
    print(f"Indices: {indices}")

    # Get the book titles and distances
    recommended_books = []
    for i, dist in zip(indices[0][1:], distances[0][1:]):  # Skip the first one as it's the book itself
        book_isbn = user_item_matrix.columns[i]
        title = df_books[df_books['isbn'] == book_isbn]['title'].values
        if len(title) > 0:
            recommended_books.append([title[0], dist])

    # Sort the books by distance and get the top 5 recommendations
    recommended_books = sorted(recommended_books, key=lambda x: x[1])[:5]

    return [book_title, recommended_books]


In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)
def test_book_recommendation():
    test_pass = True
    recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")

    # Check if the returned book title matches
    if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
        test_pass = False

    # Define expected results
    expected_recommendations = [
        ('The Lovely Bones: A Novel', 0.7234864),
        ('I Know This Much Is True', 0.76770747),
        ('The Surgeon', 0.7699411),
        ('The Weight of Water', 0.77085835),
        ("I'll Be Seeing You", 0.8016211)
    ]

    # Check if the recommended books and distances are within the expected range
    recommended_books = [rec[0] for rec in recommends[1]]
    recommended_distances = [rec[1] for rec in recommends[1]]

    expected_books = [rec[0] for rec in expected_recommendations]
    expected_distances = [rec[1] for rec in expected_recommendations]

    if recommended_books != expected_books:
        test_pass = False
    if not all(abs(r - e) < 0.05 for r, e in zip(recommended_distances, expected_distances)):
        test_pass = False

    if test_pass:
        print("You passed the challenge! 🎉🎉🎉🎉🎉")
    else:
        print("You haven't passed yet. Keep trying!")

test_book_recommendation()



Book Index: 435
Book Vector Shape: (1, 888)
Distances: [[0.         0.7234864  0.76770747 0.7699411  0.77085835 0.8016211 ]]
Indices: [[435 108  27 182 120 625]]
["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Lovely Bones: A Novel', 0.7234864], ['I Know This Much Is True', 0.76770747], ['The Surgeon', 0.7699411], ['The Weight of Water', 0.77085835], ["I'll Be Seeing You", 0.8016211]]]
Book Index: 435
Book Vector Shape: (1, 888)
Distances: [[0.         0.7234864  0.76770747 0.7699411  0.77085835 0.8016211 ]]
Indices: [[435 108  27 182 120 625]]
You passed the challenge! 🎉🎉🎉🎉🎉
